## Model Serialization notebook

In [1]:
# Importing libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

### Loading the model 

In [18]:
###  Model Blueprint
from torch import Tensor
from typing import Type

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels: int, out_channels: int, stride: int = 1, downsample: nn.Module = None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x: Tensor) -> Tensor:
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, block: Type[BasicBlock], layers: list[int], num_classes: int = 1000):
        super(ResNet, self).__init__()
        self.in_channels = 64

        # Initial convolutional layer
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Residual layers
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        # Final pooling and fully connected layer
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block: Type[BasicBlock], out_channels: int, blocks: int, stride: int = 1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride=stride, downsample=downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x: Tensor) -> Tensor:
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x
    
def ResNet18(num_classes=3):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=3)

In [23]:
from torchinfo import summary
custom_model = ResNet18(3)
dummy_input = torch.rand(64, 3, 64, 64)

summary(custom_model, input_size=(64, 3, 64, 64))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [64, 3]                   --
├─Conv2d: 1-1                            [64, 64, 32, 32]          9,408
├─BatchNorm2d: 1-2                       [64, 64, 32, 32]          128
├─ReLU: 1-3                              [64, 64, 32, 32]          --
├─MaxPool2d: 1-4                         [64, 64, 16, 16]          --
├─Sequential: 1-5                        [64, 64, 16, 16]          --
│    └─BasicBlock: 2-1                   [64, 64, 16, 16]          --
│    │    └─Conv2d: 3-1                  [64, 64, 16, 16]          36,864
│    │    └─BatchNorm2d: 3-2             [64, 64, 16, 16]          128
│    │    └─ReLU: 3-3                    [64, 64, 16, 16]          --
│    │    └─Conv2d: 3-4                  [64, 64, 16, 16]          36,864
│    │    └─BatchNorm2d: 3-5             [64, 64, 16, 16]          128
│    │    └─ReLU: 3-6                    [64, 64, 16, 16]          --
│

In [24]:
state_dict = torch.load('model/best_model.pt', map_location=torch.device('cpu'))

In [26]:
custom_model.load_state_dict(state_dict['model_state_dict'])

<All keys matched successfully>

### Serializing the model

In [27]:
traced_model = torch.jit.trace(custom_model, dummy_input)

/home/siddharthm10/.sidPyEnvs/ml-llm-env/lib/python3.10/site-packages/torch/nn/functional.py:2474: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if size_prods == 1:


### Saving the model

In [28]:
torch.jit.save(traced_model, "serialized_models/model_v1.pt")